### Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #data visualization
import datetime as dt

In [ ]:
#The parser() fun is useful when you have a string representing a date and time in a specific format
#and you want to convert it into a DateTime object for further manipulation and analysis using pandas
def parser(s):
    return pd.to_datetime(s, format='%d/%m/%Y %H:%M:%S.%f')

In [ ]:
#Meteo Data
#four quarterly data files for the year 2022 are read
LC_01 = pd.read_csv("dataverse_files/LC_2022Q1.csv")
LC_02 = pd.read_csv("dataverse_files/LC_2022Q2.csv")
LC_03 = pd.read_csv("dataverse_files/LC_2022Q3.csv")
LC_04 = pd.read_csv("dataverse_files/LC_2022Q4.csv")
Meta = pd.read_csv("dataverse_files/01_Metadata_v2.csv")

In [ ]:
#combines the data for specific IDs (LC-065, LC-087, LC-102, LC-109, LC-112, LC-117, LC-118) 
#from the quarterly meteorological DataFrames (LC_01, LC_02, LC_03, LC_04) 
#into separate DataFrames (LC_065, LC_087, LC_102, LC_109, LC_112, LC_117, LC_118)

LC_065 = pd.concat([LC_01[LC_01["ID"]=="LC-065"], LC_02[LC_02["ID"]=="LC-065"], LC_03[LC_03["ID"]=="LC-065"], LC_04[LC_04["ID"]=="LC-065"]], axis=0).reset_index(drop=True)
LC_087 = pd.concat([LC_01[LC_01["ID"]=="LC-087"], LC_02[LC_02["ID"]=="LC-087"], LC_03[LC_03["ID"]=="LC-087"], LC_04[LC_04["ID"]=="LC-087"]], axis=0).reset_index(drop=True)
LC_102 = pd.concat([LC_01[LC_01["ID"]=="LC-102"], LC_02[LC_02["ID"]=="LC-102"], LC_03[LC_03["ID"]=="LC-102"], LC_04[LC_04["ID"]=="LC-102"]], axis=0).reset_index(drop=True)
LC_109 = pd.concat([LC_01[LC_01["ID"]=="LC-109"], LC_02[LC_02["ID"]=="LC-109"], LC_03[LC_03["ID"]=="LC-109"], LC_04[LC_04["ID"]=="LC-109"]], axis=0).reset_index(drop=True)
LC_112 = pd.concat([LC_01[LC_01["ID"]=="LC-112"], LC_02[LC_02["ID"]=="LC-112"], LC_03[LC_03["ID"]=="LC-112"], LC_04[LC_04["ID"]=="LC-112"]], axis=0).reset_index(drop=True)
LC_117 = pd.concat([LC_01[LC_01["ID"]=="LC-117"], LC_02[LC_02["ID"]=="LC-117"], LC_03[LC_03["ID"]=="LC-117"], LC_04[LC_04["ID"]=="LC-117"]], axis=0).reset_index(drop=True)
LC_118 = pd.concat([LC_01[LC_01["ID"]=="LC-118"], LC_02[LC_02["ID"]=="LC-118"], LC_03[LC_03["ID"]=="LC-118"], LC_04[LC_04["ID"]=="LC-118"]], axis=0).reset_index(drop=True)

#The pd.concat() function is used to concatenate the rows with matching IDs from each quarterly DataFrame. 
#The axis=0 parameter specifies that the concatenation should be done along the rows (vertically). 
#The reset_index(drop=True) part resets the index of the concatenated DataFrame

### averaging data of 7 weather stations

In [ ]:
#create list with all dataframes
dfs = [LC_065, LC_087, LC_102, LC_109, LC_112, LC_117, LC_118]

#initialize the merged_df variable with the first DataFrame from the list, LC_065 
#and list of suffixes to be used when merging the DataFrames
merged_df = dfs[0]
suffixes = ['', '_2', '_3', '_4', '_5', '_6', '_7']

#It loops through the remaining DataFrames (dfs[1:]) and merges each DataFrame with the merged_df DataFrame 
#based on the "DATEUTC" column using an inner join. The suffixes are applied
# [index start from 1]
for i, df in enumerate(dfs[1:], start=1):
    merged_df = pd.merge(merged_df, df, on='DATEUTC', how='inner', suffixes=('', suffixes[i]))

# list of columns to average
columns_to_average = ["LC_HUMIDITY", "LC_DWPTEMP", "LC_n", "LC_RAD", "LC_RAININ", "LC_DAILYRAIN", "LC_WINDDIR", "LC_WINDSPEED", "LC_RAD60", "LC_TEMP_QCL0", "LC_TEMP_QCL1", "LC_TEMP_QCL2", "LC_TEMP_QCL3"]

#create a dictionary average_columns that maps each column name to the mean of corresponding columns with suffixes in the merged_df DataFrame
average_columns = {col: merged_df[[f"{col}{suffix}" for suffix in suffixes]].mean(axis=1) for col in columns_to_average}
# dataframe of averages
average_df = pd.DataFrame(average_columns)

#add additional columns to average_df based on the values derived from 'DATEUTC'
average_df['DATEUTC'] = merged_df['DATEUTC']
average_df["ID"] = "AVE"
average_df['Date'] = pd.to_datetime(average_df['DATEUTC']).dt.strftime('%Y/%m/%d')
average_df['Year'] = pd.to_datetime(average_df['DATEUTC']).dt.year
average_df['Month'] = pd.to_datetime(average_df['DATEUTC']).dt.month
average_df['Day'] = pd.to_datetime(average_df['DATEUTC']).dt.day
average_df['Hour'] = pd.to_datetime(average_df['DATEUTC']).dt.hour
average_df['Minute'] = pd.to_datetime(average_df['DATEUTC']).dt.minute

#reoder columns
reorder_columns = ["DATEUTC", "ID", "LC_HUMIDITY", "LC_DWPTEMP", "LC_n", "LC_RAD", "LC_RAININ", "LC_DAILYRAIN", "LC_WINDDIR", "LC_WINDSPEED", "Date", "Year", "Month", "Day", "Hour", "Minute", "LC_RAD60", "LC_TEMP_QCL0", "LC_TEMP_QCL1", "LC_TEMP_QCL2", "LC_TEMP_QCL3"]

#assign reordered average_df to the variable average_df
average_df = average_df[reorder_columns]

average_df

In [ ]:
#create csv
average_df.to_csv('LC_2022_Ave.csv')

### weather data analysit

LC_RAININ

In [ ]:
#line plot showing the "LC_RAININ" data for each  dataframe
plt.figure(figsize=(12, 6))

plt.plot(LC_065["LC_RAININ"], label="LC-065")
plt.plot(LC_087["LC_RAININ"], label="LC-087")
plt.plot(LC_102["LC_RAININ"], label="LC-102")
plt.plot(LC_109["LC_RAININ"], label="LC-109")
plt.plot(LC_112["LC_RAININ"], label="LC-112")
plt.plot(LC_117["LC_RAININ"], label="LC-117")
plt.plot(LC_118["LC_RAININ"], label="LC-118")

plt.xlabel("Index") #x axis represents index values
plt.ylabel("LC_RAININ") #y axis represents rain
plt.legend()
plt.title("LC_RAININ for 7 Dataframes")
plt.show()

In [ ]:
#combine "LC_RAININ" columns from the seven DataFrames into a new DataFrame and assign names to columns
df_combined = pd.concat([LC_065["LC_RAININ"], LC_087["LC_RAININ"], LC_102["LC_RAININ"], LC_109["LC_RAININ"], LC_112["LC_RAININ"], LC_117["LC_RAININ"], LC_118["LC_RAININ"]], axis=1)
df_combined.columns = ["LC_RAININ1", "LC_RAININ2", "LC_RAININ3", "LC_RAININ4", "LC_RAININ5", "LC_RAININ6", "LC_RAININ7"]
df_combined = df_combined.dropna() #drop missings

#concatenation along columns because axis=1

In [ ]:
#perform paired t-tests between different combinations of the "LC_RAININ" columns in the df_combined DataFrame. 
#It calculates the t-statistic and p-value for each pairwise comparison and prints the results

from itertools import combinations #iteration and combination of elements
from scipy.stats import ttest_rel #provides various statistical functions and distributions lie t-test

for combo in combinations(range(1, 8), 2): #generate all possible combinations of two numbers from the range 1 to 7 
    #t-statistic and p-value are assigned to the variables t_stat and p_value
    t_stat, p_value = ttest_rel(df_combined[f"LC_RAININ{combo[0]}"], df_combined[f"LC_RAININ{combo[1]}"]) 
    #results are printed using formatted strings to display the column names being compared
    print(f"LC_RAININ{combo[0]} vs LC_RAININ{combo[1]}: t-statistic = {t_stat}, p-value = {p_value}")

LC_TEMP_QCL3

In [ ]:
#SAME as above

plt.figure(figsize=(12, 6))

plt.plot(LC_065["LC_TEMP_QCL3"], label="LC-065")
plt.plot(LC_087["LC_TEMP_QCL3"], label="LC-087")
plt.plot(LC_102["LC_TEMP_QCL3"], label="LC-102")
plt.plot(LC_109["LC_TEMP_QCL3"], label="LC-109")
plt.plot(LC_112["LC_TEMP_QCL3"], label="LC-112")
plt.plot(LC_117["LC_TEMP_QCL3"], label="LC-117")
plt.plot(LC_118["LC_TEMP_QCL3"], label="LC-118")

plt.xlabel("Index")
plt.ylabel("LC_TEMP_QCL3")
plt.legend()
plt.title("LC_RAININ for 7 Dataframes")
plt.show()

In [ ]:
#SAME as above

df_combined = pd.concat([LC_065["LC_TEMP_QCL3"], LC_087["LC_TEMP_QCL3"], LC_102["LC_TEMP_QCL3"], LC_109["LC_TEMP_QCL3"], LC_112["LC_TEMP_QCL3"], LC_117["LC_TEMP_QCL3"], LC_118["LC_TEMP_QCL3"]], axis=1)
df_combined.columns = ["LC_TEMP_QCL31", "LC_TEMP_QCL32", "LC_TEMP_QCL33", "LC_TEMP_QCL34", "LC_TEMP_QCL35", "LC_TEMP_QCL36", "LC_TEMP_QCL37"]
df_combined = df_combined.dropna()

In [ ]:
#SAME as above

from itertools import combinations
from scipy.stats import ttest_rel

for combo in combinations(range(1, 8), 2):
    t_stat, p_value = ttest_rel(df_combined[f"LC_TEMP_QCL3{combo[0]}"], df_combined[f"LC_TEMP_QCL3{combo[1]}"])
    print(f"LC_TEMP_QCL3{combo[0]} vs LC_TEMP_QCL3{combo[1]}: t-statistic = {t_stat}, p-value = {p_value}")